In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import os

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [3]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name'])



In [4]:
mycursor = mydb.cursor()

query = "select id, industry, created_by_alias, created from jos_content where created >= '2022-01-01'"

mycursor.execute(query)
result = mycursor.fetchall()

In [5]:
# result

In [77]:
job_data = pd.DataFrame(result,columns=['jobid','industry','location','date'])

In [78]:
job_data['industry'] = job_data['industry'].str.split(',')

In [79]:
job_data = job_data.explode('industry')

In [80]:
job_data['location'] = job_data['location'].str.split(',')
job_data = job_data.explode('location')

In [81]:
job_data['location'] = job_data['location'].str.split('/')
job_data = job_data.explode('location')

In [82]:
job_data

,jobid,industry,location,date
0,1026751,22,Mumbai,2022-01-01 02:28:22
0,1026751,13,Mumbai,2022-01-01 02:28:22
0,1026751,48,Mumbai,2022-01-01 02:28:22
0,1026751,32,Mumbai,2022-01-01 02:28:22
1,1026752,2,Noida,2022-01-01 03:12:06
...,...,...,...,...
358088,1384843,40,Mumbai,2024-03-08 13:41:25
358088,1384843,40,Gujarat,2024-03-08 13:41:25
358089,1384844,8,Pune,2024-03-08 13:41:34
358089,1384844,8,Mumbai,2024-03-08 13:41:34


In [83]:
# job_data['location'] = job_data['location'].str.split('/')

In [84]:
job_data['location'].unique()


array(['Mumbai', 'Noida', 'Bangalore', ..., 'Serbia', ' Prague', ' IND'],
      dtype=object)

In [85]:
job_data

,jobid,industry,location,date
0,1026751,22,Mumbai,2022-01-01 02:28:22
0,1026751,13,Mumbai,2022-01-01 02:28:22
0,1026751,48,Mumbai,2022-01-01 02:28:22
0,1026751,32,Mumbai,2022-01-01 02:28:22
1,1026752,2,Noida,2022-01-01 03:12:06
...,...,...,...,...
358088,1384843,40,Mumbai,2024-03-08 13:41:25
358088,1384843,40,Gujarat,2024-03-08 13:41:25
358089,1384844,8,Pune,2024-03-08 13:41:34
358089,1384844,8,Mumbai,2024-03-08 13:41:34


In [86]:
job_data['date'] = pd.to_datetime(job_data['date'])

In [87]:
job_data.sort_values(['date'],inplace=True)

In [88]:
job_data

,jobid,industry,location,date
0,1026751,22,Mumbai,2022-01-01 02:28:22
0,1026751,13,Mumbai,2022-01-01 02:28:22
0,1026751,48,Mumbai,2022-01-01 02:28:22
0,1026751,32,Mumbai,2022-01-01 02:28:22
1,1026752,2,Noida,2022-01-01 03:12:06
...,...,...,...,...
358088,1384843,13,Mumbai,2024-03-08 13:41:25
358088,1384843,40,Gujarat,2024-03-08 13:41:25
358089,1384844,8,Mumbai,2024-03-08 13:41:34
358089,1384844,8,Pune,2024-03-08 13:41:34


In [89]:
job_data['date_str'] = job_data['date'].dt.strftime('%b %y')

In [90]:
# job_data.groupby('date_str')['location'].count().reset_index()

In [91]:
job_data

,jobid,industry,location,date,date_str
0,1026751,22,Mumbai,2022-01-01 02:28:22,Jan 22
0,1026751,13,Mumbai,2022-01-01 02:28:22,Jan 22
0,1026751,48,Mumbai,2022-01-01 02:28:22,Jan 22
0,1026751,32,Mumbai,2022-01-01 02:28:22,Jan 22
1,1026752,2,Noida,2022-01-01 03:12:06,Jan 22
...,...,...,...,...,...
358088,1384843,13,Mumbai,2024-03-08 13:41:25,Mar 24
358088,1384843,40,Gujarat,2024-03-08 13:41:25,Mar 24
358089,1384844,8,Mumbai,2024-03-08 13:41:34,Mar 24
358089,1384844,8,Pune,2024-03-08 13:41:34,Mar 24


In [92]:
job_data[['jobid','industry','date_str']]

,jobid,industry,date_str
0,1026751,22,Jan 22
0,1026751,13,Jan 22
0,1026751,48,Jan 22
0,1026751,32,Jan 22
1,1026752,2,Jan 22
...,...,...,...
358088,1384843,13,Mar 24
358088,1384843,40,Mar 24
358089,1384844,8,Mar 24
358089,1384844,8,Mar 24


In [93]:
pivot_industry = pd.pivot_table(job_data, 
              values='jobid', 
              index='date_str', 
              columns='industry', 
              aggfunc='count', 
              fill_value=0)

In [104]:
pivot_industry.sort_index(inplace=True)

In [106]:
pivot_industry.T

date_str,Apr 22,Apr 23,Aug 22,Aug 23,Dec 22,Dec 23,Feb 22,Feb 23,Feb 24,Jan 22,...,Mar 23,Mar 24,May 22,May 23,Nov 22,Nov 23,Oct 22,Oct 23,Sep 22,Sep 23
industry,,,,,,,,,,,,,,,,,,,,,
,0,36,0,187,0,50,2,16,40,2,...,28,13,1,71,0,66,0,93,0,151
1,623,670,585,744,626,440,581,556,555,580,...,766,109,616,795,619,509,513,525,656,703
10,272,359,263,534,350,390,240,319,472,234,...,361,155,268,536,366,363,255,366,300,434
11,545,777,418,721,481,439,529,600,519,378,...,714,121,474,780,500,373,427,520,444,664
12,1246,1351,1330,1340,1156,1374,1216,1372,1595,949,...,1405,261,1248,1213,1163,1842,803,1323,1216,1383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,68,126,36,169,37,85,22,54,246,21,...,112,83,76,155,30,40,35,35,33,143
67,260,200,223,305,187,186,230,243,241,232,...,228,68,204,288,226,192,204,170,208,252
7,3180,2576,2842,3205,2692,2243,3042,2436,2798,3114,...,2715,562,3020,3056,2763,2374,2365,2365,3004,3193


In [107]:
pivot_industry.T.to_csv('data/industry_pivot.csv')

In [108]:
pivot_location = pd.pivot_table(job_data, 
              values='jobid', 
              index='date_str', 
              columns='location', 
              aggfunc='count', 
              fill_value=0)

In [112]:
pivot_location.T.sort_index()

date_str,Apr 22,Apr 23,Aug 22,Aug 23,Dec 22,Dec 23,Feb 22,Feb 23,Feb 24,Jan 22,...,Mar 23,Mar 24,May 22,May 23,Nov 22,Nov 23,Oct 22,Oct 23,Sep 22,Sep 23
location,,,,,,,,,,,,,,,,,,,,,
,10,33,0,28,5,19,4,5,48,2,...,12,16,13,18,0,11,5,12,10,29
\t Hassan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
\t Mumbai,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Kancheepuram,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Africa,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,10,0,11,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
west india,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wet bengal,0,0,0,15,0,0,0,5,0,0,...,0,0,0,5,0,0,0,0,0,10
work from anywhere,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
pivot_location.T.to_csv('data/location_pivot.csv')

## Data Validation

In [126]:
job_data.loc[(job_data['date_str']=='Feb 24') & (job_data['industry']=='1')]

,jobid,industry,location,date,date_str
342353,1369108,1,Bangalore,2024-02-01 10:27:02,Feb 24
342353,1369108,1,Karnataka,2024-02-01 10:27:02,Feb 24
342392,1369147,1,Mumbai,2024-02-01 11:08:10,Feb 24
342393,1369148,1,Jamshedpur,2024-02-01 11:09:12,Feb 24
342398,1369153,1,Nasik,2024-02-01 11:12:32,Feb 24
...,...,...,...,...,...
355087,1381842,1,Delhi NCR,2024-02-29 16:34:20,Feb 24
355164,1381919,1,Bangalore,2024-02-29 17:59:56,Feb 24
355178,1381933,1,Gujarat,2024-02-29 18:28:04,Feb 24
355178,1381933,1,Haryana,2024-02-29 18:28:04,Feb 24


In [130]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name'])

mycursor = mydb.cursor()


query = '''SELECT count(distinct(j.id)), count(u.id), catid FROM `jos_content` as j
inner join `usr_applied_jobs_info` as u
ON j.id = u.jobid
WHERE date(j.created) >= '2022-01-01'
GROUP by year(j.created), month(j.created), j.catid'''

mycursor.execute(query)
result = mycursor.fetchall()

In [131]:
query_res = pd.DataFrame(result,columns=['job_count','user_count','category'])

In [132]:
query_res

,job_count,user_count,category
0,1,8,0
1,1398,179082,13
2,2216,314000,14
3,1209,94787,15
4,940,156643,16
...,...,...,...
225,434,220475,16
226,260,77517,17
227,185,63581,19
228,58,4622,21


In [133]:
query_res.to_csv('data/query_result.csv',index=False)